In [49]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

# Movie Review Data Cleaning

In [62]:
def read_raw(file_name, label):
  with open(file_name, 'r') as file:
    data = []
    for line in file:
        processed_line = preprocess_raw(line)
        data.append([processed_line.split(), label])
  return data

def preprocess_raw(text):
  text = text.replace('\u202f', ' ').replace('\xa0', ' ')
  out = ''
  for i, char in enumerate(text.lower()):
    if char.isalpha()==True or char==' ':
      out += char
  out=" ".join(out.split())
  out += ' .'
  return out

def build_data(direc):
  data=[]
  counter=0
  for label in ['pos', 'neg']:
    label_direc = os.path.join(direc, label)
    for filename in os.listdir(label_direc):
      if (label == 'pos'):
        data += read_raw(os.path.join(label_direc,str(filename)), 1)
      else:
        data += read_raw(os.path.join(label_direc,str(filename)), 0)
  print('Done')
  return data

In [63]:
data = build_data('./txt_sentoken')

Done


# Data to File

In [64]:
data = np.array(data)
file = open('data', 'wb')
pickle.dump(data, file)
file.close()

# Movie Review Dataset

In [65]:
class MovieReviewDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.data = dataset
        self.transform = transform
    
    def __getitem__(self, idx):
        temp = self.data[idx]
        if (self.transform):
            temp = self.transform(sample)
        
        sample = {'sentence': temp[0] ,'label': temp[1]}
        return sample
    
    def __len__(self):
        return len(self.data)            

# Load data file

In [66]:
file = open('data', 'rb')
data = pickle.load(file)
file.close()

X_train, X_test, y_train, y_test = train_test_split(data[:,0], data[:,1])
dataset = []
for i in range(len(X_train)):
    dataset.append([X_train[i], y_train[i]])
    
mr = MovieReviewDataset(dataset)
print(len(mr))


mr_dataloader = DataLoader(mr, batch_size = 16, shuffle=True)

48540


In [67]:
print(len(next(iter(mr_dataloader))['label']))

16
